In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recommender').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
# Get Data
data = spark.read.csv('movielens_ratings.csv',
                      header=True,inferSchema=True)

In [5]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show() # 1501 entries

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
# Split the data into training and test datasets
train_data, test_data = data.randomSplit([0.8,0.2])

In [16]:
# Initiate model (I removed negative values!)
als = ALS(maxIter=5,regParam=0.01,nonnegative=True,
          userCol='userId',itemCol='movieId',ratingCol='rating')

In [17]:
# Fit model
alsModel = als.fit(train_data)

In [18]:
predictions = alsModel.transform(test_data)

In [19]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   4.0|    12| 2.3634791|
|     31|   3.0|     7| 2.4941363|
|     85|   1.0|    13| 1.9903907|
|     85|   3.0|     6| 2.5250635|
|     85|   1.0|    25| 1.0253421|
|     85|   3.0|    21|  4.008762|
|     65|   1.0|    28|0.79266566|
|     65|   2.0|     5| 1.7551162|
|     53|   1.0|     9| 2.2262754|
|     53|   5.0|    21| 3.8923013|
|     78|   1.0|    27| 0.8550229|
|     78|   1.0|    13|  0.714256|
|     78|   1.0|    20| 0.8584664|
|     78|   1.0|     8| 1.4392581|
|     78|   1.0|     2| 0.7327111|
|     34|   3.0|    25|0.51514035|
|     81|   1.0|    22| 1.7688997|
|     81|   1.0|     6| 3.2594583|
|     81|   1.0|    16| 0.6767417|
|     81|   2.0|     9| 3.5814738|
+-------+------+------+----------+
only showing top 20 rows



In [20]:
# Evaluate your model
evaluator = RegressionEvaluator(metricName='rmse',
                                labelCol='rating',
                                predictionCol='prediction')

In [21]:
rmse = evaluator.evaluate(predictions)

In [22]:
print('RMSE')
print(rmse) # From 0 to 5 stars!

RMSE
1.1436973481411965


In [23]:
# Now, let's use this on a single User

In [25]:
single_user = (test_data.filter(test_data['userId']==11).
               select(['movieId','userId']))

In [26]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     10|    11|
|     13|    11|
|     16|    11|
|     27|    11|
|     39|    11|
|     41|    11|
|     48|    11|
|     59|    11|
|     62|    11|
|     71|    11|
|     76|    11|
|     79|    11|
|     88|    11|
|     89|    11|
|     94|    11|
|     97|    11|
|     99|    11|
+-------+------+



In [27]:
recommendations = alsModel.transform(single_user)

In [28]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     76|    11|  5.280543|
|     41|    11| 3.9507601|
|     79|    11| 3.6522458|
|     27|    11| 3.2996938|
|     59|    11|  3.005437|
|     94|    11|  2.660212|
|     88|    11| 2.6513643|
|     39|    11| 2.3576899|
|     48|    11| 2.2716782|
|     10|    11| 1.8175272|
|     89|    11| 1.7729615|
|     13|    11| 1.7550015|
|     62|    11| 1.4772742|
|     99|    11| 1.4469495|
|     97|    11| 1.1303542|
|     71|    11| 1.0851887|
|     16|    11| 0.7948066|
|      9|    11|0.15445098|
+-------+------+----------+

